In [7]:
import numpy as np
import scipy.io
import scipy.linalg
import sklearn.metrics
import sklearn.neighbors

def kernel(ker, X, X2, gamma):
    if not ker or ker == 'primal':
        return X
    elif ker == 'linear':
        if not X2:
            K = np.dot(X.T, X)
        else:
            K = np.dot(X.T, X2)
    elif ker == 'rbf':
        n1sq = np.sum(X ** 2, axis=0)
        n1 = X.shape[1]
        if not X2:
            D = (np.ones((n1, 1)) * n1sq).T + np.ones((n1, 1)) * n1sq - 2 * np.dot(X.T, X)
        else:
            n2sq = np.sum(X2 ** 2, axis=0)
            n2 = X2.shape[1]
            D = (np.ones((n2, 1)) * n1sq).T + np.ones((n1, 1)) * n2sq - 2 * np.dot(X.T, X)
        K = np.exp(-gamma * D)
    elif ker == 'sam':
        if not X2:
            D = np.dot(X.T, X)
        else:
            D = np.dot(X.T, X2)
        K = np.exp(-gamma * np.arccos(D) ** 2)
    return K


class JDA:
    def __init__(self, kernel_type='primal', dim=30, lamb=1, gamma=1, T=10):
        '''
        Init func
        :param kernel_type: kernel, values: 'primal' | 'linear' | 'rbf' | 'sam'
        :param dim: dimension after transfer
        :param lamb: lambda value in equation
        :param gamma: kernel bandwidth for rbf kernel
        :param T: iteration number
        '''
        self.kernel_type = kernel_type
        self.dim = dim
        self.lamb = lamb
        self.gamma = gamma
        self.T = T

    def fit_predict(self, Xs, Ys, Xt, Yt):
        '''
        Transform and Predict using 1NN as JDA paper did
        :param Xs: ns * n_feature, source feature
        :param Ys: ns * 1, source label
        :param Xt: nt * n_feature, target feature
        :param Yt: nt * 1, target label
        :return: acc, y_pred, list_acc
        '''
        list_acc = []
        X = np.hstack((Xs.T, Xt.T))
        X /= np.linalg.norm(X, axis=0)
        m, n = X.shape
        ns, nt = len(Xs), len(Xt)
        e = np.vstack((1 / ns * np.ones((ns, 1)), -1 / nt * np.ones((nt, 1))))
        C = len(np.unique(Ys))
        H = np.eye(n) - 1 / n * np.ones((n, n))

#         M = e * e.T * C
        Y_tar_pseudo = None
    
    
        for t in range(self.T):
            M = e * e.T * C
            #我认为M应该在里面计算。但是两种方法精度相似,你可以选择你认为适合的一种
            N = 0
            if Y_tar_pseudo is not None and len(Y_tar_pseudo) == nt:
                for c in range(0, C-1):
                    e = np.zeros((n,1))
                    e[np.where(Ys == c)] = 1 / len(Ys[Ys == c])
                    #len(Ys[Ys == c])和len(Ys[np.where(Ys == c)])的效果是一样的，一个是True/False，一个是返回的元组
                    indt = np.where(Y_tar_pseudo == c)
                    indt = [item + ns for item in indt]#之所以能这么用，应为Y_tar_pseudo是(Yt,)而不是(dt,1)
                    e[tuple(indt)] = -1 / len(Y_tar_pseudo[Y_tar_pseudo == c])
                    N += e.dot(e.T)
            M += N
            M = M / np.linalg.norm(M, 'fro')
            #除了M矩阵以外，计算过程与TCA类似
            K = kernel(self.kernel_type, X, None, gamma=self.gamma)
            n_eye = m if self.kernel_type == 'primal' else n
            a, b = np.linalg.multi_dot([K, M, K.T]) + self.lamb * np.eye(n_eye), np.linalg.multi_dot([K, H, K.T])
            w, V = scipy.linalg.eig(a, b)
            ind = np.argsort(w)
            A = V[:, ind[:self.dim]]
            Z = np.dot(A.T, K)
            Z /= np.linalg.norm(Z, axis=0)
            Xs_new, Xt_new = Z[:, :ns].T, Z[:, ns:].T
            
            #NN分类器
            clf = sklearn.neighbors.KNeighborsClassifier(n_neighbors=1)
            clf.fit(Xs_new, Ys.ravel())
            Y_tar_pseudo = clf.predict(Xt_new)
            acc = sklearn.metrics.accuracy_score(Yt, Y_tar_pseudo)
            list_acc.append(acc)
            print('JDA iteration [{}/{}]: Acc: {:.4f}'.format(t + 1, self.T, acc))
        return acc, Y_tar_pseudo, list_acc

In [2]:
if __name__ == '__main__':
    domains = ['amazon.mat','caltech.mat', 'dslr.mat', 'webcam.mat']
    for i in range(4):
        for j in range(4):
            if i != j:
                src, tar = 'D:/JDA/' + domains[i], 'D:/JDA/' + domains[j]
                src_domain, tar_domain = scipy.io.loadmat(src), scipy.io.loadmat(tar)
                Xs, Ys, Xt, Yt = src_domain['feas'], src_domain['label'], tar_domain['feas'], tar_domain['label']
                jda = JDA(kernel_type='primal', dim=30, lamb=1, gamma=1)
                acc, ypre, list_acc = jda.fit_predict(Xs, Ys, Xt, Yt)
                print(acc)

JDA iteration [1/10]: Acc: 0.4061
JDA iteration [2/10]: Acc: 0.4016


KeyboardInterrupt: 

In [8]:

src, tar = 'D:\FYP\IN718_2.mat', 'D:\FYP\TI64_2.mat'
src_domain, tar_domain = scipy.io.loadmat(src), scipy.io.loadmat(tar)
Xs, Ys, Xt, Yt = src_domain['feas'], src_domain['label'], tar_domain['feas'], tar_domain['label']
jda = JDA(kernel_type='primal', dim=2, lamb=1, gamma=1)
acc, ypre, list_acc = jda.fit_predict(Xs, Ys, Xt, Yt)
print(acc)

JDA iteration [1/10]: Acc: 0.9143
JDA iteration [2/10]: Acc: 0.8762
JDA iteration [3/10]: Acc: 0.8762
JDA iteration [4/10]: Acc: 0.8952
JDA iteration [5/10]: Acc: 0.8762
JDA iteration [6/10]: Acc: 0.8762
JDA iteration [7/10]: Acc: 0.8952
JDA iteration [8/10]: Acc: 0.8762
JDA iteration [9/10]: Acc: 0.8762
JDA iteration [10/10]: Acc: 0.8952
0.8952380952380953
